In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/customer_churn"
%cd "{base_folder}"

/content/drive/MyDrive/Colab Notebooks/customer_churn


In [5]:
from pathlib import Path
import os
import sqlite3
import pandas as pd
import tarfile
import urllib.request

def load_customer_data():
    print("[1] Checking for customer_churn.csv…")
    csv_path = Path(f"{base_folder}/data/Churn_Modelling.csv")

    print("[1] Loading customer_churn.csv into DataFrame…")
    return pd.read_csv(csv_path)


def build_3nf_sqlite(db_path="customer_churn.db"):
  print("=== BUILDING 3NF SQLITE DATA MODEL ===")

  print("\n[STEP 1] Loading CSV into DataFrame…")
  customer_churn = load_customer_data()
  print(f"Loaded {len(customer_churn)} rows.")

  print("\n[STEP 2] Creating 3NF DataFrames (customer_fact, gender, geography)…")
  gender = pd.DataFrame(customer_churn["Gender"].drop_duplicates().reset_index(drop=True)).reset_index()
  gender.rename(columns={"index": "gender_id"}, inplace=True)
  geography = pd.DataFrame(customer_churn["Geography"].drop_duplicates().reset_index(drop=True)).reset_index()
  geography.rename(columns={"index": "geography_id"}, inplace=True)
  customer_churn_selected = customer_churn[
  [   "CustomerId",
      "Surname",
      "CreditScore",
      "Age",
      "Geography",
      "Gender",
      "Tenure",
      "Balance",
      "NumOfProducts",
      "HasCrCard",
      "IsActiveMember",
      "EstimatedSalary",
      "Exited"
  ]
    ]
  merge_gender = customer_churn_selected.merge(gender, how="left", on="Gender")
  customer_fact = merge_gender.merge(geography, how="left", on="Geography")
  customer_fact.drop(["Gender", "Geography"], axis=1, inplace=True)

  print("3NF DataFrames created.")

  print("\n[STEP 3] Creating SQLite database and tables…")
  db_path = os.path.join(f"{base_folder}/data/", "customer_churn.db")
  if os.path.exists(db_path):
    print("Existing DB found. Removing…")
    os.remove(db_path)

  conn = sqlite3.connect(db_path)
  cur = conn.cursor()

  print("Running SQL schema creation script…")
  cur.executescript(
    """
    DROP TABLE IF EXISTS customer_fact;
    DROP TABLE IF EXISTS gender;
    DROP TABLE IF EXISTS geography;

    CREATE TABLE gender (
        gender_id  INTEGER PRIMARY KEY,
        gender    TEXT NOT NULL UNIQUE
    );

    CREATE TABLE geography (
        geography_id    INTEGER PRIMARY KEY,
        geography      TEXT NOT NULL UNIQUE
    );

    CREATE TABLE customer_fact (
        customerId INTEGER NOT NULL ,
        surname TEXT   NOT NULL ,
        creditScore INTEGER NOT NULL ,
        age INTEGER NOT NULL ,
        tenure INTEGER NOT NULL ,
        balance INTEGER NOT NULL ,
        numofProducts INTEGER NOT NULL ,
        hasCrCard INTEGER NOT NULL ,
        isActiveMember INTEGER NOT NULL ,
        estimatedSalary INTEGER NOT NULL ,
        exited INTEGER NOT NULL,
        gender_id INTEGER NOT NULL ,
        geography_id INTEGER NOT NULL ,
        FOREIGN KEY (geography_id)
            REFERENCES geography(geography_id),
        FOREIGN KEY (gender_id)
            REFERENCES gender(gender_id)
    );
    """
  )
  print("Tables created.")

  print("\n[STEP 4] Inserting data into SQLite database…")
  print("Inserting gender dimension…")
  cur.executemany(
    "INSERT INTO gender (gender_id, gender) VALUES (?, ?)",
    list(gender.itertuples(index=False, name=None)),
  )

  print("Inserting geography table…")
  cur.executemany(
    """
    INSERT INTO geography (geography_id, geography)
    VALUES (?, ?)
    """,
    list(geography.itertuples(index=False, name=None)),
  )

  print("Inserting customer_fact")
  cur.executemany(
    """
    INSERT INTO customer_fact (
        customerId,
        surname,
        creditScore,
        age,
        tenure,
        balance,
        numofProducts,
        hasCrCard,
        isActiveMember,
        estimatedSalary,
        exited,
        gender_id,
        geography_id
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    list(customer_fact.itertuples(index=False, name=None)),
  )

  conn.commit()
  conn.close()

  print("\n=== DONE! SQLite DB created at:", db_path, "===\n")


build_3nf_sqlite("customer_churn.db")


=== BUILDING 3NF SQLITE DATA MODEL ===

[STEP 1] Loading CSV into DataFrame…
[1] Checking for customer_churn.csv…
[1] Loading customer_churn.csv into DataFrame…
Loaded 10000 rows.

[STEP 2] Creating 3NF DataFrames (customer_fact, gender, geography)…
3NF DataFrames created.

[STEP 3] Creating SQLite database and tables…
Running SQL schema creation script…
Tables created.

[STEP 4] Inserting data into SQLite database…
Inserting gender dimension…
Inserting geography table…
Inserting customer_fact

=== DONE! SQLite DB created at: /content/drive/MyDrive/Colab Notebooks/customer_churn/data/customer_churn.db ===

